<a href="https://colab.research.google.com/github/TheRadDani/TensorFlow-functional-api/blob/main/TFModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
inputs = tf.keras.Input(shape=(3,))
x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
inputs = keras.Input(shape=(None, None, 3))
processed = keras.layers.RandomCrop(width=32, height=32)(inputs)
conv = keras.layers.Conv2D(filters=2, kernel_size=3)(processed)
pooling = keras.layers.GlobalAveragePooling2D()(conv)
feature = keras.layers.Dense(10)(pooling)

full_model = keras.Model(inputs, feature)
backbone = keras.Model(processed, conv)
activations = keras.Model(conv, feature)

Subclassing the model

In [ ]:
import tensorflow as tf

class MyModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)

  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(x)

model = MyModel()

In [ ]:
import tensorflow as tf

class MyModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)
    self.dropout = tf.keras.layers.Dropout(0.5)

  def call(self, inputs, training=False):
    x = self.dense1(inputs)
    if training:
      x = self.dropout(x, training=training)
    return self.dense2(x)

model = MyModel()

In [47]:
import numpy as np
inputs = tf.keras.layers.Input(shape=(3,))
outputs = tf.keras.layers.Dense(2)(inputs)
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="Adam", loss="mse", metrics=["mae"],jit_compile=True)
batch_size = 3000 # Set the batch size
history = model.fit(np.random.rand(batch_size,3),np.zeros((batch_size,2)),epochs = 10, verbose=2)
model.metrics_names

Epoch 1/10
94/94 - 3s - loss: 0.0927 - mae: 0.2453 - 3s/epoch - 27ms/step
Epoch 2/10
94/94 - 0s - loss: 0.0713 - mae: 0.2154 - 222ms/epoch - 2ms/step
Epoch 3/10
94/94 - 0s - loss: 0.0562 - mae: 0.1914 - 209ms/epoch - 2ms/step
Epoch 4/10
94/94 - 0s - loss: 0.0439 - mae: 0.1692 - 216ms/epoch - 2ms/step
Epoch 5/10
94/94 - 0s - loss: 0.0339 - mae: 0.1487 - 212ms/epoch - 2ms/step
Epoch 6/10
94/94 - 0s - loss: 0.0259 - mae: 0.1298 - 241ms/epoch - 3ms/step
Epoch 7/10
94/94 - 0s - loss: 0.0195 - mae: 0.1126 - 208ms/epoch - 2ms/step
Epoch 8/10
94/94 - 0s - loss: 0.0145 - mae: 0.0971 - 313ms/epoch - 3ms/step
Epoch 9/10
94/94 - 0s - loss: 0.0106 - mae: 0.0831 - 245ms/epoch - 3ms/step
Epoch 10/10
94/94 - 0s - loss: 0.0076 - mae: 0.0706 - 211ms/epoch - 2ms/step


['loss', 'mae']

In [ ]:
import tensorflow as tf

inputs = tf.keras.layers.Input(shape=(10,))
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = tf.keras.losses.BinaryCrossentropy()(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Generate 1000 samples of input data with 10 features each
x_train = np.random.randn(1000, 10)

# Generate 1000 random binary labels for the classification task
y_train = np.random.randint(2, size=1000)

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)

optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')

epochs = 10
for epoch in range(epochs):
    for step, (inputs, labels) in enumerate(train_dataset):
        train_step(inputs, labels)
    print('Epoch: {}, Loss: {}, Accuracy: {}'.format(epoch+1, train_loss.result(), train_accuracy.result()))
    train_loss.reset_states()
    train_accuracy.reset_states()


Epoch: 1, Loss: 0.696258544921875, Accuracy: 0.527999997138977
Epoch: 2, Loss: 0.682414174079895, Accuracy: 0.5600000023841858
Epoch: 3, Loss: 0.6750593781471252, Accuracy: 0.5929999947547913
Epoch: 4, Loss: 0.6698946952819824, Accuracy: 0.5979999899864197
Epoch: 5, Loss: 0.6649953722953796, Accuracy: 0.6019999980926514
Epoch: 6, Loss: 0.65837162733078, Accuracy: 0.6290000081062317
Epoch: 7, Loss: 0.651261568069458, Accuracy: 0.6359999775886536
Epoch: 8, Loss: 0.653340756893158, Accuracy: 0.6140000224113464
Epoch: 9, Loss: 0.644133985042572, Accuracy: 0.652999997138977
Epoch: 10, Loss: 0.6406399607658386, Accuracy: 0.6470000147819519


In [ ]:
subset = train_dataset.take(1)

# Convert the dataset to a NumPy array
x, y = next(iter(subset))
x = x.numpy()
y = y.numpy()
print(x.shape)
print(y.size)

(32, 10)
32


In [13]:
class MyModel(tf.keras.Model):

  def __init__(self, *args, **kwargs):
    super(MyModel, self).__init__(*args, **kwargs)
    self.loss_tracker = tf.keras.metrics.Mean(name='loss')

  def compute_loss(self, x, y, y_pred, sample_weight):
    l2_loss = tf.reduce_sum([tf.nn.l2_loss(w) for w in self.trainable_variables])
    loss = tf.reduce_mean(tf.math.squared_difference(y_pred, y)) + l2_loss
    loss += tf.add_n(self.losses)
    self.loss_tracker.update_state(loss)
    return loss

  def reset_metrics(self):
    self.loss_tracker.reset_states()

  @property
  def metrics(self):
    return [self.loss_tracker]

tensors = tf.random.uniform((10, 10)), tf.random.uniform((10,))
dataset = tf.data.Dataset.from_tensor_slices(tensors).repeat().batch(1)

inputs = tf.keras.layers.Input(shape=(10,), name='my_input')
outputs = tf.keras.layers.Dense(10)(inputs)
model = MyModel(inputs, outputs)
model.add_loss(tf.reduce_sum(outputs))

optimizer = tf.keras.optimizers.SGD()
model.compile(optimizer, loss='mse', steps_per_execution=10)
model.fit(dataset, epochs=2, steps_per_epoch=10)
print('My custom loss: ', model.loss_tracker.result().numpy())
[x.result() for x in model.metrics]

Epoch 1/2
10/10 [==============================] - 1s 78ms/step - loss: 5.6903
Epoch 2/2
10/10 [==============================] - 0s 2ms/step - loss: 2.3269
My custom loss:  2.326882


[<tf.Tensor: shape=(), dtype=float32, numpy=2.326882>]